<a href="https://colab.research.google.com/github/Karansingh-25/fastapi/blob/main/personality_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [2]:
!wget 'https://raw.githubusercontent.com/Karansingh-25/fastapi/main/dataset/personality_dataset.csv'

--2025-06-05 12:11:03--  https://raw.githubusercontent.com/Karansingh-25/fastapi/main/dataset/personality_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 107234 (105K) [text/plain]
Saving to: ‘personality_dataset.csv’

personality_dataset 100%[===================>] 104.72K  --.-KB/s    in 0.007s  

2025-06-05 12:11:03 (13.9 MB/s) - ‘personality_dataset.csv’ saved [107234/107234]



In [3]:
data = pd.read_csv('personality_dataset.csv')

In [4]:
data.head()

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,4.0,No,4.0,6.0,No,13.0,5.0,Extrovert
1,9.0,Yes,0.0,0.0,Yes,0.0,3.0,Introvert
2,9.0,Yes,1.0,2.0,Yes,5.0,2.0,Introvert
3,0.0,No,6.0,7.0,No,14.0,8.0,Extrovert
4,3.0,No,9.0,4.0,No,8.0,5.0,Extrovert


In [5]:
print(data['Drained_after_socializing'].isna().sum())


52


In [6]:
for col in data.columns:
    print(col)
    print(data[col].unique())
    print('--------------------------')

Time_spent_Alone
[ 4.  9.  0.  3.  1.  2. 10.  6.  5.  8. nan  7. 11.]
--------------------------
Stage_fear
['No' 'Yes' nan]
--------------------------
Social_event_attendance
[ 4.  0.  1.  6.  9.  7.  8.  3.  5.  2. 10. nan]
--------------------------
Going_outside
[ 6.  0.  2.  7.  4.  5. nan  3.  1.]
--------------------------
Drained_after_socializing
['No' 'Yes' nan]
--------------------------
Friends_circle_size
[13.  0.  5. 14.  8.  6.  7. 15.  4. 10.  1. 12.  2. 11.  9.  3. nan]
--------------------------
Post_frequency
[ 5.  3.  2.  8.  6.  7.  0. 10.  4.  1.  9. nan]
--------------------------
Personality
['Extrovert' 'Introvert']
--------------------------


# Data Cleaning

In [7]:
data1=data.copy()

In [8]:
data1.head()

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,4.0,No,4.0,6.0,No,13.0,5.0,Extrovert
1,9.0,Yes,0.0,0.0,Yes,0.0,3.0,Introvert
2,9.0,Yes,1.0,2.0,Yes,5.0,2.0,Introvert
3,0.0,No,6.0,7.0,No,14.0,8.0,Extrovert
4,3.0,No,9.0,4.0,No,8.0,5.0,Extrovert


In [9]:
categorical=['Stage_fear','Drained_after_socializing']
numeric=['Time_spent_Alone','Friends_circle_size','Social_event_attendance','Going_outside','Post_frequency']

In [10]:
for col in categorical:
    print(col)
    data1[col].fillna('No',inplace=True)

Stage_fear
Drained_after_socializing


<ipython-input-10-61f9e9c583d7>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data1[col].fillna('No',inplace=True)


In [11]:
for col in numeric:
    print(col)
    data1[col].fillna(0,inplace=True)

Time_spent_Alone
Friends_circle_size
Social_event_attendance
Going_outside
Post_frequency


<ipython-input-11-6e3ac68c0c4f>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data1[col].fillna(0,inplace=True)


In [12]:
for col in data1.columns:
    print(col)
    print(data1[col].isna().sum())
    print('--------------------------')

Time_spent_Alone
0
--------------------------
Stage_fear
0
--------------------------
Social_event_attendance
0
--------------------------
Going_outside
0
--------------------------
Drained_after_socializing
0
--------------------------
Friends_circle_size
0
--------------------------
Post_frequency
0
--------------------------
Personality
0
--------------------------


In [13]:
data1.shape

(2900, 8)

In [14]:
for col in data1.columns:
    print(col)
    print(data1[col].unique())
    print('--------------------------')

Time_spent_Alone
[ 4.  9.  0.  3.  1.  2. 10.  6.  5.  8.  7. 11.]
--------------------------
Stage_fear
['No' 'Yes']
--------------------------
Social_event_attendance
[ 4.  0.  1.  6.  9.  7.  8.  3.  5.  2. 10.]
--------------------------
Going_outside
[6. 0. 2. 7. 4. 5. 3. 1.]
--------------------------
Drained_after_socializing
['No' 'Yes']
--------------------------
Friends_circle_size
[13.  0.  5. 14.  8.  6.  7. 15.  4. 10.  1. 12.  2. 11.  9.  3.]
--------------------------
Post_frequency
[ 5.  3.  2.  8.  6.  7.  0. 10.  4.  1.  9.]
--------------------------
Personality
['Extrovert' 'Introvert']
--------------------------


# Feature Engineering

In [15]:
#1.time_spent_alone

def private_time(Time_spent_Alone):
    if Time_spent_Alone < 4:
        return "Low"
    elif Time_spent_Alone >=4 and Time_spent_Alone < 8:
        return "Medium"
    else :
        return "High"

In [16]:
data1["private_time"] = data1["Time_spent_Alone"].apply(private_time)

In [17]:
data1['private_time'].head(5)

,private_time
0,Medium
1,High
2,High
3,Low
4,Low


In [18]:
# 2. social activity score
def activity_score(row):
    if row['Social_event_attendance']+row['Going_outside']+row['Post_frequency'] <=7:
        return "Low"
    elif row['Social_event_attendance']+row['Going_outside']+row['Post_frequency'] >=8 and row['Social_event_attendance']+row['Going_outside']+row['Post_frequency'] <=14:
        return "Medium"
    else:
        return "High"

In [19]:
data1["activity_score"] = data1.apply(activity_score, axis=1)

In [20]:
data1['activity_score'].head(5)

,activity_score
0,High
1,Low
2,Low
3,High
4,High


In [21]:
# 3. friend circle size
def making_friends(Friends_circle_size):
    if Friends_circle_size < 4:
        return "Low"
    elif Friends_circle_size >=4 and Friends_circle_size < 9:
        return "Medium"
    else :
        return "High"

In [51]:
data1['making_friends'] = data1['Friends_circle_size'].apply(making_friends)

In [52]:
data1['making_friends'].head(5)

,making_friends
0,High
1,Low
2,Medium
3,High
4,Medium


In [53]:
data.columns

Index(['Time_spent_Alone', 'Stage_fear', 'Social_event_attendance',
       'Going_outside', 'Drained_after_socializing', 'Friends_circle_size',
       'Post_frequency', 'Personality'],
      dtype='object')

In [54]:
data1.drop(columns=['Time_spent_Alone', 'Social_event_attendance',
       'Going_outside', 'Friends_circle_size'])[['private_time','activity_score','Post_frequency','making_friends','Personality']].head(5)

,private_time,activity_score,Post_frequency,making_friends,Personality
0,Medium,High,5.0,High,Extrovert
1,High,Low,3.0,Low,Introvert
2,High,Low,2.0,Medium,Introvert
3,Low,High,8.0,High,Extrovert
4,Low,High,5.0,Medium,Extrovert


In [55]:
categorical_for_transformer=['private_time','activity_score','making_friends']
numeric_for_transformer=['Post_frequency']

In [56]:
X=data1[['private_time','activity_score','Post_frequency','making_friends']]
y=data1['Personality']

In [57]:
X

,private_time,activity_score,Post_frequency,making_friends
0,Medium,High,5.0,High
1,High,Low,3.0,Low
2,High,Low,2.0,Medium
3,Low,High,8.0,High
4,Low,High,5.0,Medium
...,...,...,...,...
2895,Low,High,6.0,Medium
2896,Low,High,9.0,High
2897,Medium,Low,0.0,Medium
2898,High,Low,0.0,Low


In [58]:
y

,Personality
0,Extrovert
1,Introvert
2,Introvert
3,Extrovert
4,Extrovert
...,...
2895,Extrovert
2896,Extrovert
2897,Introvert
2898,Introvert


# Dataset Preprocessing

In [59]:
transformer=ColumnTransformer(transformers=[
    ("cat",OneHotEncoder(),categorical_for_transformer),
    ("num",'passthrough',numeric_for_transformer)
])

In [60]:
pipeline=Pipeline([
    ('pipe1',transformer),
    ('pipe2',RandomForestClassifier(random_state=42))
])

# Split Dataset

In [61]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [62]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((2320, 4), (580, 4), (2320,), (580,))

In [63]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('pipe1',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  ['private_time',
                                                   'activity_score',
                                                   'making_friends']),
                                                 ('num', 'passthrough',
                                                  ['Post_frequency'])])),
                ('pipe2', RandomForestClassifier(random_state=42))])

In [64]:
y_pred=pipeline.predict(X_test)
accuracy_score(y_test,y_pred)

0.9293103448275862

# Save The Model

In [65]:
import pickle

pickle_model_path='model.pkl'
with open(pickle_model_path,'wb') as f:
    pickle.dump(pipeline,f)